In [ ]:
!pip install langchain langchain_community language-tool-python langchain-ollama jq langchain_huggingface langchain_chroma faiss-cpu

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
import csv
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
import json
from langchain_chroma import Chroma

# 문서 로드

In [ ]:
def load_example_sentences(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    return [Document(page_content=sentence) for sentence in data]


example_sentences = load_example_sentences("./extracted_examples.json")

example_sentences

[Document(metadata={}, page_content='Let me introduce my aunt.'),
 Document(metadata={}, page_content='I am going to play soccer.'),
 Document(metadata={}, page_content='The boy is my brother.'),
 Document(metadata={}, page_content='Jimmy is going to exercise every day.'),
 Document(metadata={}, page_content='The girl is my sister.'),
 Document(metadata={}, page_content='Let me introduce my cousin to you.'),
 Document(metadata={}, page_content='We are going to learn Korean history.'),
 Document(metadata={}, page_content='I’m going to run a marathon.'),
 Document(metadata={}, page_content='He is going to visit his grandpa.'),
 Document(metadata={}, page_content='The woman is my mom.'),
 Document(metadata={}, page_content='He set a new world record for swimming.'),
 Document(metadata={}, page_content='Just sit down and relax for an hour.'),
 Document(metadata={}, page_content='He repairs computers.'),
 Document(metadata={}, page_content='The teacher repeated the question.'),
 Document(me

# 모델 생성

In [29]:
# llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 500)
llm = ChatOllama(model="llama3.2", max_token=50, temperature=0.5)

# 프롬프트 

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    You must create exactly one simple, complete English sentence that an elementary school student can understand.
    
    - The sentence must include the word '{query}' exactly as specified, without any modifications or additional forms.
    - Use only a single statement, not a question, command, or compound sentence.
    - The sentence should be between 5 and 8 words long.
    - Do not create multiple sentences or add any additional information.
    - Use only common, everyday words that are simple for an elementary school student in the US to understand.

    If you cannot create a sentence following these rules exactly, respond with "Unable to create a valid sentence."
    """,
)

# # 프롬프트 템플릿 설정
# prompt_template = PromptTemplate(
#     input_variables=["query"],
#     template="""CREATE ONE EASY SENTENCE FOR CHILDREN:

# RULES:
# 1. Use '{query}' in your sentence
# 2. Use 5-8 simple words only
# 3. Make it easy for grade 1-3 students
# 4. NO hard or complex words
# 5. NO questions or long sentences
# 6. NO extra sentences or examples
# 7. WRITE ONLY ONE SENTENCE

# Examples of RIGHT LEVEL:
# - "The dog plays in the park"
# - "My mom bakes cookies for me"

# >>> Write only ONE simple sentence:"""
# )

# 임베딩 생성

In [ ]:
# embeddings = OpenAIEmbeddings()
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "sentence-transformers/all-mpnet-base-v2"


# model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu", "trust_remote_code": True},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

# vectoc store

In [10]:
# 벡터스토어 저장
vector_store = FAISS.from_documents(example_sentences, hf_embeddings)
# 생성된 벡터 스토어를 디스크에 저장
vector_store.save_local("./sentence_vectorstore")

# 저장할 경로 지정
DB_PATH = "./sentence_vectorstore"

# # DB 생성
# db = Chroma.from_documents(
#     documents=example_sentences, embedding=hf_embeddings, collection_name="sentence_db",persist_directory=DB_PATH
# )

In [ ]:
# # 디스크에서 문서를 로드합니다.
# DB_PATH = "./sentence_vectorstore"

# chroma_db = Chroma(
#     persist_directory=DB_PATH,
#     embedding_function=hf_embeddings,
#     collection_name="sentence_db",
# )

In [ ]:
# 저장된 벡터 스토어 로드
loaded_vector_store = FAISS.load_local(
    "./sentence_vectorstore",
    hf_embeddings,
    allow_dangerous_deserialization=True,  # 신뢰된 파일에서만 사용
)

# 검색기 생성

In [ ]:
retriever = loaded_vector_store.as_retriever()
# retriever = chroma_db.as_retriever()

In [32]:
retriever.invoke("eraser")

[Document(metadata={}, page_content='It’s her eraser.'),
 Document(metadata={}, page_content='Did you find the eraser?'),
 Document(metadata={}, page_content='It’s your pencil.'),
 Document(metadata={}, page_content='Whose paintbrush is it?')]

# 체인 생성

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "query"},
)

In [34]:
prompt_template

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\n    You must create exactly one simple, complete English sentence that an elementary school student can understand.\n    \n    - The sentence must include the word \'{query}\' exactly as specified, without any modifications or additional forms.\n    - Use only a single statement, not a question, command, or compound sentence.\n    - The sentence should be between 5 and 8 words long.\n    - Do not create multiple sentences or add any additional information.\n    - Use only common, everyday words that are simple for an elementary school student in the US to understand.\n\n    If you cannot create a sentence following these rules exactly, respond with "Unable to create a valid sentence."\n    ')

# 실행 코드(테스트)

In [35]:
# JSON 파일에서 단어 목록 불러오기
with open("./extracted_words.json", "r", encoding="utf-8") as file:
    words = json.load(file)

In [ ]:
import csv
from tqdm import tqdm


# CSV 파일 생성
with open("grammar_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["문장 번호", "문장", "포함된 단어"])

    # 각 단어에 대해 문장 생성 및 저장
    for idx, word in tqdm(enumerate(words, start=1), disable=False):
        response = qa_chain.invoke({"query": word})  # 체인 실행

        # response의 형식이 예상대로 되어 있는지 확인
        if "result" in response:
            generated_sentence = response["result"]

            # 문장에 포함된 단어가 정확히 있는지 검증
            if word in generated_sentence:
                writer.writerow([idx, generated_sentence, word])
                print(generated_sentence, word)
            else:
                print(f"생성된 문장이 규칙을 따르지 않습니다. 단어: {word}")
        else:
            print(f"응답 형식이 잘못되었습니다. 단어: {word}")

1it [00:04,  4.78s/it]

생성된 문장이 규칙을 따르지 않습니다. 단어: attractive


2it [00:05,  2.32s/it]

Ryan is very handsome. handsome


3it [00:06,  1.58s/it]

...

생성된 문장이 규칙을 따르지 않습니다. 단어: occur


1938it [27:20,  1.08it/s]

생성된 문장이 규칙을 따르지 않습니다. 단어: appear


1939it [27:21,  1.18it/s]

He has a talent for acting. act
